## Data Mining project: Discover and describe areas of interest and events from geo-located data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium

In [2]:
data_path="flickr_data2.csv"
df=pd.read_csv(data_path)
df.head(n=5)

C:\Users\abouziane\AppData\Local\Temp\ipykernel_10612\3555437615.py:2: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(data_path)


,id,user,lat,long,tags,title,date_taken_minute,date_taken_hour,date_taken_day,date_taken_month,date_taken_year,date_upload_minute,date_upload_hour,date_upload_day,date_upload_month,date_upload_year,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,4395181099,30624617@N03,45.754858,4.821710,"chair,lyon,rhône,chaise,rhônealpes",Chaises avec vue,11.0,15,28,2,2010,23,20,28.0,2,2010.0,NaN,NaN,NaN
1,4394748717,35853470@N00,45.753270,4.862953,NaN,NaN,51.0,17,28,2,2010,52,17,28.0,2,2010.0,NaN,NaN,NaN
2,4394694699,11817998@N05,45.760655,4.846564,"365,iphone",59/365 - R46 V103 B163,29.0,17,28,2,2010,33,17,28.0,2,2010.0,NaN,NaN,NaN
3,4394803790,11545749@N06,45.784000,4.874072,"nin,nineinchnails,gift,screening,toiou,avott",2010-01-29 Toiou Avott Lyon,15.0,20,28,1,2010,38,12,28.0,2,2010.0,NaN,NaN,NaN
4,4394803554,11545749@N06,45.784000,4.874072,"lyon,nin,nineinchnails,gift,screening,toiou,avott",2010-01-28 Toiou Avott Lyon,10.0,20,28,1,2010,38,12,28.0,2,2010.0,NaN,NaN,NaN


## Discover the data

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420240 entries, 0 to 420239
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   420240 non-null  int64  
 1    user                420240 non-null  object 
 2    lat                 420240 non-null  float64
 3    long                420240 non-null  float64
 4    tags                316730 non-null  object 
 5    title               381911 non-null  object 
 6    date_taken_minute   420239 non-null  float64
 7    date_taken_hour     420240 non-null  int64  
 8    date_taken_day      420240 non-null  int64  
 9    date_taken_month    420240 non-null  int64  
 10   date_taken_year     420240 non-null  int64  
 11   date_upload_minute  420228 non-null  object 
 12   date_upload_hour    420238 non-null  object 
 13   date_upload_day     420238 non-null  float64
 14   date_upload_month   420240 non-null  int64  
 15   date_upload_year

We notice that the Unnamed columns are all almost empty ,but are filled when data is incoherent . Finally , we decided to just eraze the rows where Unnamed columns are defined , and then we dropped the three columns .

In [4]:
df = df.drop_duplicates(subset=['id'], keep='first')
df = df[df[['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18']].isnull().all(axis=1)]


df.drop(columns=['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18'],inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 168050 entries, 0 to 419137
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   168050 non-null  int64  
 1    user                168050 non-null  object 
 2    lat                 168050 non-null  float64
 3    long                168050 non-null  float64
 4    tags                126069 non-null  object 
 5    title               152273 non-null  object 
 6    date_taken_minute   168050 non-null  float64
 7    date_taken_hour     168050 non-null  int64  
 8    date_taken_day      168050 non-null  int64  
 9    date_taken_month    168050 non-null  int64  
 10   date_taken_year     168050 non-null  int64  
 11   date_upload_minute  168050 non-null  object 
 12   date_upload_hour    168050 non-null  object 
 13   date_upload_day     168050 non-null  float64
 14   date_upload_month   168050 non-null  int64  
 15   date_upload_year    1

We also thought about working on data points located in the Lyon region in a defined radius . We define this zone by       

lat_min, lat_max = 45.65, 45.85


lon_min, lon_max = 4.75, 4.95

In [5]:


# Define the bounding box coordinates for the Lyon region
lat_min, lat_max = 45.65, 45.85
lon_min, lon_max = 4.75, 4.95

# Calculate the center of the bounding box
center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2

# Create a map centered on the Lyon region
lyon_map = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Define the coordinates for the bounding box
bounding_box_coords = [
    [lat_min, lon_min],  # Bottom-left
    [lat_min, lon_max],  # Bottom-right
    [lat_max, lon_max],  # Top-right
    [lat_max, lon_min],  # Top-left
    [lat_min, lon_min]   # Close the polygon
]

# Add the bounding box to the map
folium.PolyLine(bounding_box_coords, color='blue', weight=2).add_to(lyon_map)

# Display the map
lyon_map.save("lyon_region_map.html")
lyon_map  # This will render the map in a Jupyter Notebook environment if supported



In [6]:


# Filter the DataFrame based on latitude and longitude ranges
filtered_df = df[(df[' lat'] >= lat_min) & (df[' lat'] <= lat_max) &
                 (df[' long'] >= lon_min) & (df[' long'] <= lon_max)]

# Display the filtered DataFrame
filtered_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 161654 entries, 0 to 419137
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   161654 non-null  int64  
 1    user                161654 non-null  object 
 2    lat                 161654 non-null  float64
 3    long                161654 non-null  float64
 4    tags                121622 non-null  object 
 5    title               146375 non-null  object 
 6    date_taken_minute   161654 non-null  float64
 7    date_taken_hour     161654 non-null  int64  
 8    date_taken_day      161654 non-null  int64  
 9    date_taken_month    161654 non-null  int64  
 10   date_taken_year     161654 non-null  int64  
 11   date_upload_minute  161654 non-null  object 
 12   date_upload_hour    161654 non-null  object 
 13   date_upload_day     161654 non-null  float64
 14   date_upload_month   161654 non-null  int64  
 15   date_upload_year    1

In [10]:
from sklearn.cluster import KMeans, kmeans_plusplus

# Define the number of clusters
num_clusters = 3

# Extract coordinates for clustering
coords = filtered_df[[' lat', ' long']].to_numpy()  # Ensure column names match exactly

# Use kmeans_plusplus to initialize cluster centers
init_centers, _ = kmeans_plusplus(coords, n_clusters=num_clusters, random_state=42)

# Perform KMeans clustering with the initialized centers
kmeans = KMeans(n_clusters=num_clusters, init=init_centers, n_init=1, random_state=42)
filtered_df['cluster'] = kmeans.fit_predict(coords).astype(int)

# Print the resulting DataFrame
filtered_df


C:\Users\abouziane\AppData\Local\Temp\ipykernel_10612\3653563263.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['cluster'] = kmeans.fit_predict(coords).astype(int)


,id,user,lat,long,tags,title,date_taken_minute,date_taken_hour,date_taken_day,date_taken_month,date_taken_year,date_upload_minute,date_upload_hour,date_upload_day,date_upload_month,date_upload_year,cluster
0,4395181099,30624617@N03,45.754858,4.821710,"chair,lyon,rhône,chaise,rhônealpes",Chaises avec vue,11.0,15,28,2,2010,23,20,28.0,2,2010.0,0
1,4394748717,35853470@N00,45.753270,4.862953,NaN,NaN,51.0,17,28,2,2010,52,17,28.0,2,2010.0,2
2,4394694699,11817998@N05,45.760655,4.846564,"365,iphone",59/365 - R46 V103 B163,29.0,17,28,2,2010,33,17,28.0,2,2010.0,0
3,4394803790,11545749@N06,45.784000,4.874072,"nin,nineinchnails,gift,screening,toiou,avott",2010-01-29 Toiou Avott Lyon,15.0,20,28,1,2010,38,12,28.0,2,2010.0,2
4,4394803554,11545749@N06,45.784000,4.874072,"lyon,nin,nineinchnails,gift,screening,toiou,avott",2010-01-28 Toiou Avott Lyon,10.0,20,28,1,2010,38,12,28.0,2,2010.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413844,48994955623,23723681@N05,45.785599,4.813400,NaN,20190918_155502,55.0,15,18,9,2019,13.0,13,1.0,11,2019.0,0
413845,48995498101,23723681@N05,45.784841,4.812855,NaN,20190918_155720,57.0,15,18,9,2019,13.0,13,1.0,11,2019.0,0
413846,48994653443,128086472@N06,45.781863,4.875416,NaN,L’égout et les couleurs,42.0,16,30,10,2019,33.0,11,1.0,11,2019.0,2
419071,44824515765,14869313@N00,45.768700,4.840502,"france,auvergnerhônealpes,auvergnerhonealpes,r...","Rhône river, Lyon, France",41.0,11,27,10,2018,9.0,22,5.0,11,2018.0,0
